In [ ]:
%pip install plotly scipy ipywidgets

# Comparisons tool 2: Reducing distances from video to video

This tool aims to give to a tournesol user a list of suggested comparisons.

- Comparisons between 2 videos marked as "Public" (will not suggest comparisons with "Private" videos)
- Compute the distance between compared videos, by the minimum number of comparisons to cross to get from one to the other
- Find good new comparisons to reduce these distances as much as possible

Requirements:
- Knowing how to get Authentication JWT from Tournesol website
- Having more than 20 compared videos (With few videos, this tool is not useful and may not suggest any comparisons)

Special rules:

- Will not suggest to add comparisons to videos where there are less contributors than your comparisons with it\
  (Prefer improving popular videos to improve global Tournesol connectivity as much as possible)

In [ ]:
# Imports
import os
import math
import time
import random
import networkx as nx
import plotly.graph_objects as go
from datetime import datetime, timezone

# Ensure notebook is running from src/ dir
_pwd = os.path.realpath('.').split(os.sep)
if 'src' in _pwd:
	while _pwd[-1] != 'src':
		_pwd.pop()
	os.chdir(os.sep.join(_pwd))
print(os.path.realpath('.'))

# Local project requirements
from model.tournesol_api import TournesolAPI, get, get_individual_score

## Parameters

JWT: Get it from tournesol.app

- open [website](https://tournesol.app), authenticate, then open dev tools, get any request to Tournesol api, see Request Headers, get `Authentication="Bearer ..."` value
- DO NOT SHARE THIS TOKEN TO ANYONE. NEVER. IN ANY CONDITIONS. It's sensitive like your password.
- This token expires after some time of inactivity (about a day or so). If tool fails, try to update the token first.

In [ ]:
# PARAMETERS
TOURNESOL_API=TournesolAPI(input('JWT (example: "Bearer xxxxxxxxx")'))
TOURNESOL_API.loadCache(f"../data/Tournesol_API_cache-{TOURNESOL_API.username}.json.gz")

MAX_PUBLIC_COMPARISONS = 10 # will not suggest to compare videos having more than this
MAX_NUMBER_OF_SUGGESTIONS = 25 # will generate this amount of suggested comparisons

In [ ]:
# Load dataset
TOURNESOL_API.getMyComparedVideos(useCache=False, saveCache=True) # Put video data in cache
comparisons = TOURNESOL_API.getAllMyComparisons()

public_undgraph = nx.Graph()
private_digraph = nx.DiGraph()
for cdata in comparisons:
	if cdata.get('is_public', False):
		public_undgraph.add_edge(cdata['entity_a'], cdata['entity_b'])

	score = [dta['score'] for dta in cdata['criteria_scores'] if dta['criteria'] == 'largely_recommended'][0]
	if score >= 0:
		private_digraph.add_edge(cdata['entity_a'], cdata['entity_b'])
	if score <= 0:
		private_digraph.add_edge(cdata['entity_b'], cdata['entity_a'])
private_undgraph = private_digraph.to_undirected(as_view=True)

videos = {vid: TOURNESOL_API.getVData(vid, useCache=True, saveCache=False) for vid in private_undgraph.nodes}
for vid,video in videos.items():
	if get(video, False, 'individual_rating', 'is_public') and (vid not in public_undgraph):
		public_undgraph.add_node(vid)

print('Videos', len(videos))
print('Comparisons', len(comparisons))
print('Public', public_undgraph)
print('Private', private_undgraph)
print('Directed', private_digraph)

In [ ]:
# Suggest comparisons
centernode = max(public_undgraph.nodes, key=lambda v: private_undgraph.degree[v])
contribs = {vid:get(videos[vid], 0, 'collective_rating', 'n_contributors') for vid in public_undgraph.nodes}
candidates = {vid:public_undgraph.degree[vid] for vid in public_undgraph.nodes
	if (
		# number of contributors > public comparisons
		contribs[vid] > private_undgraph.degree[vid]
		# Or if has no connexion to highest degree node
		or not nx.has_path(private_undgraph, vid, centernode)
	)
	# days since last comparisons >= public comparisons (x public comparisons: will not be suggested for x days after a comparison)
	and (datetime.now(timezone.utc) - datetime.fromisoformat(get(videos[vid], '2000-01-01T00:00:00Z', 'individual_rating', 'last_compared_at'))).days >= public_undgraph.degree[vid]
}

# Get all candidates scores
indiv_score = {vid:get_individual_score(videos[vid]) or 0 for vid in candidates}

MAXD_PRIVATE = private_undgraph.number_of_nodes()
MAXD_PUBLIC = public_undgraph.number_of_nodes()

sim_public_undgraph = public_undgraph.copy()
sim_private_digraph = private_digraph.copy()
sim_private_undgraph = sim_private_digraph.to_undirected(as_view=True)

suggested = 0

while suggested < MAX_NUMBER_OF_SUGGESTIONS and len(candidates) > 1:
	distances:dict[str,dict[str,tuple[int,int]]] = {}
	dist_public:dict[str,int] = {}
	dist_private:dict[str,int] = {}
	clist = sorted(candidates)
	for c in clist:
		private_dists_from_c = nx.single_source_shortest_path_length(sim_private_undgraph, source=c)
		public_dists_from_c = nx.single_source_shortest_path_length(sim_public_undgraph, source=c)
		distances[c] = {}
		dist_public[c] = 0
		dist_private[c] = 0
		for d in clist:
			if d >= c: break
			prv_dist = private_dists_from_c.get(d, MAXD_PRIVATE)
			pub_dist = public_dists_from_c.get(d, MAXD_PUBLIC)
			dist_private[c] += prv_dist
			dist_private[d] += prv_dist
			dist_public[c] += pub_dist
			dist_public[d] += pub_dist
			distances[c][d] = (pub_dist, prv_dist)
			distances[d][c] = (pub_dist, prv_dist)

	if not distances:
		break

	# Get as the first video, the one that:
	# - (1) has the more distance to others in public comparisons
	# - (2) has the more distance to others in private comparisons
	# - (3) has the largest difference of contributors - indiv_public_comparisons
	cmp1 = max(distances, key=lambda c1: (dist_public.get(c1), dist_private.get(c1), contribs[c1] - sim_public_undgraph.degree[c1]))

	# Second candidate may not be connected to first candidate in directed graph
	connected_nodes = {cmp1, *nx.descendants(sim_private_digraph, cmp1), *nx.ancestors(sim_private_digraph, cmp1)}
	candidates_2 = list(vid for vid in candidates if vid not in connected_nodes)
	if candidates_2:
		# Get as the second video, the one that:
		# - (1) has the more distance to cmp1 in public graph
		# - (2) has the more distance to cmp2 in private graph
		# - (3) has the indiv_score the most similar to cmp1
		cmp2 = max(candidates_2, key=lambda c2: (distances[cmp1].get(c2), -abs(indiv_score[c2] - indiv_score[cmp1])))

		# Shuffle left&right
		(cmp1, cmp2) = random.choice([(cmp1, cmp2), (cmp2, cmp1)])
		sp1 = '∞'
		sp2 = '∞'
		try: sp1 = nx.shortest_path_length(sim_private_undgraph, cmp1, cmp2)
		except: pass
		try: sp2 = nx.shortest_path_length(sim_public_undgraph, cmp1, cmp2)
		except: pass
		def _pubpriv(pub, priv):
			if pub == priv:
				if pub == '∞':
					return ' ∞    '
				return f"{pub:2d}    "
		
			if pub == '∞':
				return f" {pub}({priv:2d})"
			
			return f"{pub:2d}({priv-pub:+1d})"

		suggested += 1
		print(f"{suggested:4d}: https://tournesol.app/comparison?uidA={cmp1}&uidB={cmp2} dist:{_pubpriv(sp2,sp1)} cmps:{_pubpriv(public_undgraph.degree[cmp1], private_undgraph.degree[cmp1])} & {_pubpriv(public_undgraph.degree[cmp2], private_undgraph.degree[cmp2])}")
		sim_private_digraph.add_edge(cmp1, cmp2)
		sim_private_digraph.add_edge(cmp2, cmp1)
		sim_public_undgraph.add_edge(cmp1, cmp2)
		candidates.pop(cmp2)
	candidates.pop(cmp1)

-----

# Graph

Draw publicaly compared videos (as colored circles) and the comparisons between them.

Positions does not have meaning. Videos move around to try to untangle the graph as much as possible (compared videos nearer, and others further away).

Color Legend:
- Dark Blue: Central videos (with few jumps from comparison to comparison, rapidly access every other videos)
- Blue: Well connected videos
- Green~Yellow: Average
- Orange: Distant videos, more comparisons may be needed
- Red: Most distant videos (Most of the others videos have a high number of jumps from comparison to comparison to get to it)

Note: Distante calculation (& color) take into account private comparisons, even if private videos arn't displayed (a blue or green node may be rendered far away from the center for example if it is compared with 1 comparison to a private video connected to a central video)

In [ ]:
# Display graph
def pos_to_graphlocs(public_graph:nx.Graph, pos:dict[str,list[int]]):
	nodes = {'x': [], 'y': [], 'l': []}
	for node in public_graph:
		if node in pos:
			x, y = pos[node]
			nodes['l'].append(node)
			nodes['x'].append(x)
			nodes['y'].append(y)

	edges = {'x': [], 'y': []}
	for edge in public_graph.edges():
		if not (edge[0] in pos and edge[1] in pos): continue
		x0, y0 = pos[edge[0]]
		x1, y1 = pos[edge[1]]
		
		if len(edges['x']) > 2 and edges['x'][-2] == x0 and edges['y'][-2] == y0:
			edges['x'].insert(-1, x1)
			edges['y'].insert(-1, y1)
		elif len(edges['x']) > 2 and edges['x'][-2] == x1 and edges['y'][-2] == y1:
			edges['x'].insert(-1, x0)
			edges['y'].insert(-1, y0)
		else:
			edges['x'].append(x0)
			edges['x'].append(x1)
			edges['y'].append(y0)
			edges['y'].append(y1)
			edges['x'].append(None)
			edges['y'].append(None)

	return {
		'nodes': nodes,
		'edges': edges
	}


MAX_CONNECTED_GRAPH = public_undgraph.subgraph(max(nx.connected_components(public_undgraph), key=len))
def init_comparisons_graph(public_graph:nx.Graph, private_graph:nx.Graph):
	pos = nx.spring_layout(MAX_CONNECTED_GRAPH, pos=nx.circular_layout(MAX_CONNECTED_GRAPH), iterations=10)
	loc = pos_to_graphlocs(public_graph, pos)

	scatters = []
	# Public edges
	scatters.append(go.Scatter(
		x=loc['edges']['x'], y=loc['edges']['y'],
		line=dict(
			width=0.3,
			color='#888',
		),
		hoverinfo='none',
		mode='lines',
		showlegend=False,
	))

	# Nodes text & colors
	cnct:dict[str,float] = dict()
	mx = private_graph.number_of_nodes()
	for n1,tgt in nx.all_pairs_shortest_path_length(private_graph, cutoff=32):
		ttl = mx
		for n2,ln in tgt.items():
			if n2 == n1: continue
			ttl -= 1/ln
		cnct[n1] = mx/(mx - ttl) # Color: Average distance to every other nodes of the graph

	node_colors = []
	node_text = []
	for node in loc['nodes']['l']:
		deg_pub = public_graph.degree[node] if node in public_graph else 0
		deg_prv = private_graph.degree[node]
		node_text.append(
			f"{node}<br>"
			+ f"{deg_pub} public comparisons<br>"
			+ (f"{deg_prv - deg_pub} private comparisons<br>" if deg_prv > deg_pub else "<br>")
			+ f"Average distance: {cnct[node]:.1f}"
		)
		node_colors.append(cnct[node])

	ticks = list(range(int(min(node_colors)+1), int(max(node_colors)+1)))
	if min(node_colors) <= ticks[0] - .1:
		ticks.insert(0, min(node_colors))
	if max(node_colors) >= ticks[-1] + .1:
		ticks.append(max(node_colors))

	# Public nodes
	scatters.append(go.Scatter(
		x=loc['nodes']['x'], y=loc['nodes']['y'],
		mode='markers',
		hoverinfo='text',
		marker=dict(
			colorscale='Portland',
			colorbar=dict(
				title='Avg. distance<br>to others',
				tickformat='.1f',
				tickvals=ticks,
				ticks='outside',
				tickmode='array',
			),
			reversescale=False,
			size=3,
			line=dict(width=0),
			color=node_colors,
		),
		text=node_text,
		showlegend=True,
	))

	fig = go.FigureWidget(data=scatters,
		layout=go.Layout(
			plot_bgcolor='white',
			showlegend=False,
			hovermode='closest',
			margin=dict(b=0,l=0,r=0,t=0),
			xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
			yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
			height=720,
			width=1080,
		),
	)

	# Fix aspect ratio
	fig.update_yaxes(scaleanchor="x", scaleratio=1)

	return fig,pos

fig,pos = init_comparisons_graph(public_undgraph, private_undgraph)
fig # Display graph (should be the last line of the notebook cell)

In [ ]:
# Untangle graph above (may run multiple time if needed - Takes time)
TIME_TO_RUN = 120 # Seconds, the longer the prettier
STEP_TIME = 5 # Seconds

def improve_graph_pos(time_to_run:int, pos:dict[str,list[float]], callback=None, target_refresh_interval:int=None):
	start = time.time()

	target_total_it_count=math.ceil(time_to_run/target_refresh_interval) if target_refresh_interval > 0 else 10
	iterations_count=10
	total_iterations=0
	loops_count = 0
	#layout = ForceLayout(MAX_CONNECTED_GRAPH)
	#layout.update_graph(pos, edge_lengths=lambda e: 1)
	timer_a = time.time()
	while timer_a - start < time_to_run:
		loops_count += 1
		# Move nodes towards eachother if connected, move them apart from eachother if not connected
		pos = nx.spring_layout(MAX_CONNECTED_GRAPH, pos=pos, iterations=iterations_count, center=[0,0], weight=None)
		#layout.iterate2(repulsion_factor=0.01, repulse_upper_bound=3, inertia_factor=.9)
		total_iterations += iterations_count
		timer_b = time.time()
		speed = iterations_count / (timer_b-timer_a)
		expected_remaining_iterations = speed * (time_to_run - timer_b + start)
		if callback:
			#pos = layout.get_pos()
			callback(pos)
		print(f"Iterations: {total_iterations}/{total_iterations + expected_remaining_iterations:.0f} -- Time: {timer_b-start:.1f}/{time_to_run}s -- Speed: {speed:.1f}/s")
		next_iteration_count = int(math.ceil(expected_remaining_iterations / (target_total_it_count - loops_count if loops_count < target_total_it_count else 1)))
		if loops_count > target_total_it_count or next_iteration_count > iterations_count*2 and loops_count > 1:
			# Spring Layout may stop iterating if found an equilibrium. Try to detect this event and stop before max_duration
			break
		# Prepare next iteration
		iterations_count = next_iteration_count
		timer_a = timer_b

	return pos
	#return layout.get_pos()

def onupdate(pos):
	loc = pos_to_graphlocs(public_undgraph, pos)
	with fig.batch_update():
		#Public Mixed Private Nodes
		fig.data[0]['x'] = loc['edges']['x']
		fig.data[0]['y'] = loc['edges']['y']
		fig.data[1]['x'] = loc['nodes']['x']
		fig.data[1]['y'] = loc['nodes']['y']

pos = improve_graph_pos(TIME_TO_RUN, pos, callback=onupdate, target_refresh_interval=STEP_TIME)

In [ ]:
# 3D Graph Display

def pos_to_graphlocs_3d(public_graph:nx.Graph, pos3d:dict[str,list[int]]):
	nodes = {'x': [], 'y': [], 'z': [], 'l': []}
	for node in public_graph:
		if node in pos3d:
			x, y, z = pos3d[node]
			nodes['l'].append(node)
			nodes['x'].append(x)
			nodes['y'].append(y)
			nodes['z'].append(z)

	edges = {'x': [], 'y': [], 'z': []}
	for edge in public_graph.edges():
		if not (edge[0] in pos3d and edge[1] in pos3d): continue
		x0, y0, z0 = pos3d[edge[0]]
		x1, y1, z1 = pos3d[edge[1]]
		
		if len(edges['x']) > 2 and edges['x'][-2] == x0 and edges['y'][-2] == y0 and edges['z'][-2] == z0:
			edges['x'].insert(-1, x1)
			edges['y'].insert(-1, y1)
			edges['z'].insert(-1, z1)
		elif len(edges['x']) > 2 and edges['x'][-2] == x1 and edges['y'][-2] == y1 and edges['z'][-2] == z0:
			edges['x'].insert(-1, x0)
			edges['y'].insert(-1, y0)
			edges['z'].insert(-1, z0)
		else:
			edges['x'].append(x0)
			edges['y'].append(y0)
			edges['z'].append(z0)
			edges['x'].append(x1)
			edges['y'].append(y1)
			edges['z'].append(z1)
			edges['x'].append(None)
			edges['y'].append(None)
			edges['z'].append(None)

	return {
		'nodes': nodes,
		'edges': edges
	}

def init_comparisons_graph(public_graph:nx.Graph, private_graph:nx.Graph):
	pos3d = nx.spring_layout(MAX_CONNECTED_GRAPH, dim=3, iterations=10)
	loc3d = pos_to_graphlocs_3d(public_graph, pos3d)

	scatters = []
	# Public edges
	scatters.append(go.Scatter3d(
		x=loc3d['edges']['x'], y=loc3d['edges']['y'], z=loc3d['edges']['z'],
		line=dict(
			width=0.3,
			color='#888',
		),
		hoverinfo='none',
		mode='lines',
	))

	# Nodes text & colors
	cnct:dict[str,float] = dict()
	mx = private_graph.number_of_nodes()
	for n1,tgt in nx.all_pairs_shortest_path_length(private_graph, cutoff=32):
		ttl = mx
		for n2,ln in tgt.items():
			if n2 == n1: continue
			ttl -= 1/ln
		cnct[n1] = mx/(mx - ttl)

	node_colors = []
	node_text = []
	for node in loc3d['nodes']['l']:
		deg_pub = public_graph.degree[node] if node in public_graph else 0
		deg_prv = private_graph.degree[node]
		node_text.append(
			f"{node}<br>"
			+ f"{deg_pub} public comparisons<br>"
			+ (f"{deg_prv - deg_pub} private comparisons<br>" if deg_prv > deg_pub else "<br>")
			+ f"Average distance: {cnct[node]:.1f}"
		)
		node_colors.append(cnct[node])

	# Public nodes
	scatters.append(go.Scatter3d(
		x=loc3d['nodes']['x'], y=loc3d['nodes']['y'], z=loc3d['nodes']['z'],
		mode='markers',
		hoverinfo='text',
		marker=dict(
			colorscale='Portland',
			colorbar=dict(
				title='Avg. distance<br>to others',
				tickformat='.1f',
				tickvals=[min(node_colors), *range(int(min(node_colors)+1), int(max(node_colors)+1)), max(node_colors)],
				ticks='outside',
				tickmode='array',
			),
			reversescale=False,
			color=node_colors,
			size=3,
			line=dict(width=0),
		),
		text=node_text,
	))

	fig = go.FigureWidget(data=scatters,
		layout=go.Layout(
			showlegend=False,
			hovermode='closest',
			margin=dict(b=0,l=0,r=0,t=0),
			height=720,
			width=1080,
		),
	)

	fig.update_layout(scene=dict(
		xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
		yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
		zaxis=dict(showgrid=False, zeroline=False, showticklabels=False, visible=False),
	))

	# Fix aspect ratio
	fig.update_yaxes(scaleanchor="x", scaleratio=1)

	return fig,pos3d

fig,pos3d = init_comparisons_graph(public_undgraph, private_undgraph)
fig # Display graph (should be the last line of the notebook cell)

In [ ]:
# Untangle 3D Graph above (may run multiple time if needed - Takes time)
TIME_TO_RUN = 120 # Seconds, the longer the prettier
STEP_TIME = 5 # Seconds

def improve_graph_pos3d(time_to_run:int, pos3d:dict[str,list[float]], callback=None, target_refresh_interval:int=None):
	start = time.time()

	target_total_it_count=math.ceil(time_to_run/target_refresh_interval) if target_refresh_interval > 0 else 10
	iterations_count=10
	total_iterations=0
	loops_count = 0
	timer_a = time.time()
	while timer_a - start < time_to_run:
		loops_count += 1
		# Move nodes towards eachother if connected, move them apart from eachother if not connected
		pos3d = nx.spring_layout(MAX_CONNECTED_GRAPH, dim=3, pos=pos3d, iterations=iterations_count, center=[0,0,0], weight=None)
		total_iterations += iterations_count
		timer_b = time.time()
		speed = iterations_count / (timer_b-timer_a)
		expected_remaining_iterations = speed * (time_to_run - timer_b + start)
		if callback:
			callback(pos3d)
		print(f"Iterations: {total_iterations}/{total_iterations + expected_remaining_iterations:.0f} -- Time: {timer_b-start:.1f}/{time_to_run}s -- Speed: {speed:.1f}/s")
		next_iteration_count = int(math.ceil(expected_remaining_iterations / (target_total_it_count - loops_count if loops_count < target_total_it_count else 1)))
		if loops_count > target_total_it_count or next_iteration_count > iterations_count*2 and loops_count > 1:
			# Spring Layout may stop iterating if found an equilibrium. Try to detect this event and stop before max_duration
			break
		# Prepare next iteration
		iterations_count = next_iteration_count
		timer_a = timer_b

	return pos3d

def onupdate3d(pos3d):
	loc = pos_to_graphlocs_3d(public_undgraph, pos3d)
	with fig.batch_update():
		fig.data[0]['x'] = loc['edges']['x']
		fig.data[0]['y'] = loc['edges']['y']
		fig.data[0]['z'] = loc['edges']['z']
		fig.data[1]['x'] = loc['nodes']['x']
		fig.data[1]['y'] = loc['nodes']['y']
		fig.data[1]['z'] = loc['nodes']['z']

pos3d = improve_graph_pos3d(TIME_TO_RUN, pos3d, callback=onupdate3d, target_refresh_interval=STEP_TIME)